# not used

# 0. 환경설정

In [53]:
!pip install praat-parselmouth

In [54]:
import parselmouth
from parselmouth.praat import call
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import google.colab.drive as drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# 1. Extract and Modulate Pitch Contour: Using Threshold

In [113]:
# pitch 추출

def extract_pitch_values(audio_file):
    sound = parselmouth.Sound(audio_file)
    pitch = sound.to_pitch_ac(time_step=0.01, pitch_floor=50, pitch_ceiling=500) # time_step=0.01 (default)

    # Extract raw pitch values and timestamps
    pitch_values = pitch.selected_array['frequency']
    time_stamps = pitch.xs()

    # Filter out unvoiced segments
    valid_indices = pitch_values > 0
    pitch_values = pitch_values[valid_indices]
    time_stamps = time_stamps[valid_indices]

    return pitch_values, time_stamps

In [130]:
def get_pitch_over_threshold(audio_file, threshold=10.0):  # threshold=10으로 고정
    """
    Extract stylized pitch features including movement distance and slope from an audio file.
    """
    pitch_values, time_stamps = extract_pitch_values(audio_file)

    if len(pitch_values) == 0:  # 값 없으면 빈 리스트 반환 후 호출 종료
        print(f"No valid pitch values found in {audio_file}")
        return []

    # Apply manual stylization
    applied_pitch = [pitch_values[0]]  # 첫 음높이 값 포함
    applied_time = [time_stamps[0]]   # 첫 시간 포함

    for i in range(1, len(pitch_values)):
        # 이전 값과 현재 값의 차이가 threshold보다 크면 추가
        if abs(pitch_values[i] - applied_pitch[-1]) >= threshold:
            applied_pitch.append(pitch_values[i])
            applied_time.append(time_stamps[i])

    applied_pitch, applied_time = np.array(applied_pitch), np.array(applied_time)  # array

    # Visualize the results
    # visualize_pitch_values(applied_pitch, applied_time, title="Stylized Pitch Values")

    # Calculate movement distance and slope
    contour_data = [] # 시간, 음높이 저장 리스트
    for i in range(1, len(applied_pitch)):
        contour_data.append({'time': applied_time[i], 'pitch': applied_pitch[i]}) # dic -> df

    return contour_data  # type == list, dic -> df 변환 필요

def get_pitch_contour_threshold_process(input_folder, output_file, threshold=10):
    """
    Processes all .wav files in a folder, extracts pitch features, and saves them to a CSV file.
    - input_folder: Path to the folder containing .wav files
    - output_file: Path to save the output CSV file
    - threshold: Threshold for pitch stylization
    """
    f0_contour_values = []  # List to store features from all files

    for file_name in os.listdir(input_folder):
        if file_name.endswith(".wav"):
            audio_file = os.path.join(input_folder, file_name)
            utterance_contour = get_pitch_over_threshold(audio_file, threshold)

            # Convert to DataFrame if it's a list  # error 해결 위해 추가
            if isinstance(utterance_contour, list):
                utterance_contour = pd.DataFrame(utterance_contour)

            if not utterance_contour.empty:
                utterance_contour['file_name'] = file_name  # Add file name column
                f0_contour_values.append(utterance_contour)  # Append DataFrame

    if f0_contour_values:
        # Combine all DataFrames into one
        combine = pd.concat(f0_contour_values, ignore_index=True)
        save_to_csv(combine, output_file)
        print(f"All features saved to {output_file}.")  # Debugging
        return combine
    else:
        print("No pitch data was extracted.")
        return pd.DataFrame()

# 2. Generate Pitch Movement Slope Data
- Intonation Curve Standardization
- the physical feature of pitch range, moving time, moving distance, slope

**Reference**
  - Jeahyuk Oh. (2014). *A Study of Methods of Standardization for Korean Intonation Curve*. 한국어학, 62, 395-420.
  - Jeahyuk Oh. (2024).*Improving the objectivity of intonation transcription*. 한말연구, 65(25), 1-20.

In [135]:
def calculate_movement_distance_and_slope(pitch_values, time_stamps):
    """
    음높이 움직임의 이동 거리와 기울기를 계산합니다.
    """
    # 음높이 값을 0~100 사이로 정규화합니다.
    min_pitch = np.min(pitch_values)  # 음높이의 최솟값
    max_pitch = np.max(pitch_values)  # 음높이의 최댓값
    normalized_pitch = (pitch_values - min_pitch) / (max_pitch - min_pitch) * 100

    # 시간 값을 0~100 사이로 정규화합니다.
    min_time = np.min(time_stamps)
    max_time = np.max(time_stamps)
    normalized_time = (time_stamps - min_time) / (max_time - min_time) * 100

    movement_data = []  # 이동 거리와 기울기를 저장할 리스트
    for i in range(1, len(normalized_pitch)):
        delta_pitch = normalized_pitch[i] - normalized_pitch[i - 1]  # 음높이 차이
        delta_time = normalized_time[i] - normalized_time[i - 1]  # 시간 차이
        distance = np.sqrt(delta_pitch**2 + delta_time**2)  # 피타고라스 정리로 이동 거리 계산
        slope = delta_pitch / delta_time if delta_time != 0 else 0  # 기울기 계산
        movement_data.append({
            'start_time': normalized_time[i - 1],
            'end_time': normalized_time[i],
            'start_pitch': normalized_pitch[i - 1],
            'end_pitch': normalized_pitch[i],
            'distance': distance,
            'slope': slope
        })

    return movement_data

def extract_pitch_features_with_stylization_and_distance(audio_file, threshold=10.0):  # threshold 통한 pitch point 조정 과정 포함됨
    """
    Extract stylized pitch features including movement distance and slope from an audio file.
    """
    pitch_values, time_stamps = extract_pitch_values(audio_file)

    if len(pitch_values) == 0:
        print(f"No valid pitch values found in {audio_file}")
        return []

    # Apply manual stylization
    stylized_pitch = [pitch_values[0]]  # 첫 음높이 값 포함
    stylized_time = [time_stamps[0]]   # 첫 시간 포함

    for i in range(1, len(pitch_values)):
        # 이전 값과 현재 값의 차이가 threshold보다 크면 추가
        if abs(pitch_values[i] - stylized_pitch[-1]) >= threshold:
            stylized_pitch.append(pitch_values[i])
            stylized_time.append(time_stamps[i])

    stylized_pitch, stylized_time = np.array(stylized_pitch), np.array(stylized_time)

    # Visualize the results
    # visualize_pitch_values(pitch_values, time_stamps, title="Raw Pitch Values")
    # visualize_pitch_values(stylized_pitch, stylized_time, title="Stylized Pitch Values")

    # Calculate movement distance and slope
    movement_data = calculate_movement_distance_and_slope(stylized_pitch, stylized_time)

    return movement_data

In [136]:
def get_pitch_movement_process(input_folder, threshold=10, output_csv="/content/pitch_movement_data.csv"):
    """
    Processes all .wav files in a folder, extracts pitch features, and saves them to a CSV file.
    - threshold: Threshold for pitch stylization
    - output_csv: Output file name for the processed data
    """
    pitch_movement_data = []

    for file_name in os.listdir(input_folder):
        if file_name.endswith(".wav"):  # Process only .wav files
            audio_file = os.path.join(input_folder, file_name)
            try:
                utterance_movement = extract_pitch_features_with_stylization_and_distance(audio_file, threshold)

                if not utterance_movement:
                    print(f"No pitch data for {file_name}, skipping.")
                    continue

                # Add a column to identify the source file
                for entry in utterance_movement:
                    entry['file_name'] = file_name

                pitch_movement_data.extend(utterance_movement)

                print(f"Processed {file_name}.")
            except Exception as e:
                print(f"Error processing {file_name}: {e}")

    # Convert the list of dictionaries to a DataFrame
    df_movement = pd.DataFrame(pitch_movement_data)

    # Save to CSV
    if not df_movement.empty:
        df_movement.to_csv(output_csv, index=False)
        print(f"Saved pitch movement data to {output_csv}")
        return df_movement
    else:
        print("No valid data to save.")
        return pd.DataFrame()

In [96]:
def merge_pitch_movement_data(df, full_data):
    df_pitch_movement = df.copy()

    # 불러온 데이터에서 필요한 행만 필터링
    matching_filenames = df_pitch_movement['file_name'].unique()
    filtered_data = full_data[full_data['voice_piece_filename'].isin(matching_filenames)]

    # 필요한 컬럼만 유지
    columns_to_keep = ['voice_piece_filename', 'styles', 'emotions', 'gender', 'age', 'disagree']
    filtered_data = filtered_data[columns_to_keep]

    # 원본 데이터프레임과 필터링된 데이터프레임 결합
    merged_data = df_pitch_movement.merge(filtered_data, left_on='file_name', right_on='voice_piece_filename', how='inner')

    return merged_data

In [97]:
def group_pitch_data(df):
    """
    Groups pitch data by 'file_name'.
    - df: DataFrame containing pitch features
    - Returns: Grouped object
    """
    if not df.empty:
        return df.groupby('file_name')
    else:
        print("DataFrame is empty. Cannot group data.")
        return None

# 3. Intonation Curve Standardization

In [98]:
def stylize_pitch_tier_from_sound(audio_file, frequency_resolution=2.0, pitch_floor=50, pitch_ceiling=500):
    """
    음성 파일에서 PitchTier를 생성하고 유형화 합니다.
    """
    try:
        sound = parselmouth.Sound(audio_file)

        # PitchTier로 변환
        manipulation = call(sound, "To Manipulation", 0.01, pitch_floor, pitch_ceiling)
        pitch_tier = call(manipulation, "Extract pitch tier")

        # 근접 복사 유형화 수행
        call(pitch_tier, "Stylize...", frequency_resolution, "semitones")

        # 결과값 추출
        # PitchTire 객체에서는 값 추출이 불가능함 (API 제공 X). Praat의 Synthesize > "To Pitch..." 이용하여 다시 pitch 객체로 변환
        close_copy_pitch = call(pitch_tier, "To Pitch...", 0.01, pitch_floor, pitch_ceiling)
        stylized_pitch = close_copy_pitch.selected_array['frequency']
        stylized_time = close_copy_pitch.xs()

        if len(stylized_time) == 0 or len(stylized_pitch) == 0:
            raise ValueError("Stylized time or pitch is empty.")

        return np.array(stylized_time), np.array(stylized_pitch)

    except Exception as e:
        print(f"Failed to process {audio_file}: {e}")
        return np.array([]), np.array([])

def calculate_slopes(stylized_time, stylized_pitch):
    """
    유형화 된 Pitch 데이터를 바탕으로 각 구간의 기울기를 계산합니다.
    """
    slopes = []
    for i in range(1, len(stylized_time)):
        delta_pitch = stylized_pitch[i] - stylized_pitch[i - 1]
        delta_time = stylized_time[i] - stylized_time[i - 1]
        slope = delta_pitch / delta_time if delta_time != 0 else 0
        slopes.append(slope)
    return np.array(slopes)


def plot_pitch_stylization(time_stamps, pitch_values, stylized_time, stylized_pitch, output_path=None):
    """
    음높이 곡선과 근접 복사 유형화 결과를 시각화합니다.
    - time_stamps: 원래의 시간 값 배열
    - pitch_values: 원래의 음높이 값 배열
    - stylized_time: 스타일화된 시간 값 배열
    - stylized_pitch: 스타일화된 음높이 값 배열
    - output_path: 그래프 저장 경로 (None이면 저장하지 않음)
    """
    plt.figure(figsize=(12, 6))

    # 원 음높이 곡선
    plt.plot(time_stamps, pitch_values, label="Original Pitch Contour", color="gray", linestyle="--")

    # 스타일화된 음높이 곡선
    plt.plot(stylized_time, stylized_pitch, label="Stylized Pitch Contour", color="green", marker="o", linewidth=2)

    # 그래프 설정
    plt.xlabel("Time (s)")
    plt.ylabel("Pitch (Hz)")
    plt.title("Close-Copy Stylization")
    plt.ylim(50, 500) # 음역대와 일치
    plt.legend()
    plt.grid(True)

    # 그래프 저장 또는 표시
    if output_path:
        plt.savefig(output_path)
        print(f"Plot saved to {output_path}")
    plt.show()

def save_stylized_pitch(stylized_time, stylized_pitch, slopes, output_file):
    """
    유형화 된 음높이 데이터를 CSV 파일로 저장합니다.
    """
    # slopes의 길이를 stylized_time과 맞추기 위해 None 값을 추가
    if len(slopes) < len(stylized_time):
        slopes = list(slopes) + [None]  # 마지막 구간의 기울기 값 없음 처리

    data = {
        "Time (s)": stylized_time,
        "Pitch (Hz)": stylized_pitch,
        "Slope": slopes
    }
    df = pd.DataFrame(data)
    df.to_csv(output_file, index=False)
    print(f"Stylized pitch data saved to {output_file}")

def get_pitch_stylized_data(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for file_name in os.listdir(input_folder):
        if file_name.endswith(".wav"):
            sound_file = os.path.join(input_folder, file_name)

            base_name = os.path.splitext(file_name)[0]
            output_plot = os.path.join(output_folder, f"{base_name}_plot.png")
            output_csv = os.path.join(output_folder, f"{base_name}_pitch.csv")

            try:
                # PitchTier 유형화 수행
                stylized_time, stylized_pitch = stylize_pitch_tier_from_sound(sound_file, frequency_resolution=2.0)

                # 각 구간의 기울기 계산
                slopes = calculate_slopes(stylized_time, stylized_pitch)

                # 시각화
                plot_pitch_stylization(stylized_time, stylized_pitch, stylized_time, stylized_pitch, output_path=output_plot)

                # 결과 저장
                save_stylized_pitch(stylized_time, stylized_pitch, slopes, output_csv)

                print(f"Processed {file_name}: Plot saved to {output_plot}, CSV saved to {output_csv}.")
            except Exception as e:
                print(f"Failed to process {file_name}: {e}")

# **Baseline**

In [131]:
# data
input_folder = "/content/drive/MyDrive/AIFFELthon/Data/Sample/literature_100/sample" # Speech corpus (Raw data)
drive_csv_path = "/content/drive/MyDrive/AIFFELthon/Data/Literature/label_lit.csv" # Labeling data
full_data = pd.read_csv(drive_csv_path)

output_contour = "/content/pitch_contour_data.csv"
output_movement = "/content/pitch_movement_data.csv"
output_stylization_folder = "/content/processed_results_res2.0"  # 근접복사유형화 결과 파일을 저장할 폴더

In [139]:
def entry_point() :
    # Data 1: threshold
    df_pitch_countour = get_pitch_contour_threshold_process(input_folder, output_contour, threshold=10) 

    # Data 2: pitch movement slope 
    df_pitch_movement = get_pitch_movement_process(input_folder, threshold=10, output_csv=output_movement)

    # Data 3: Stylized data
    get_pitch_stylized_data(input_folder, output_stylization_folder) 

    return df_pitch_countour, df_pitch_movement

In [140]:
if __name__ == "__main__":
    entry_point()

Processed S0171-09-11-05.wav.
Processed S0171-09-11-08.wav.
Processed S0171-09-11-07.wav.
Processed S0171-09-11-10.wav.
Processed S0171-09-11-09.wav.
Processed S0171-09-11-04.wav.
Processed S0171-09-11-03.wav.
Processed S0171-09-11-01.wav.
Processed S0171-09-11-06.wav.
Processed S0171-09-11-02.wav.
Processed S0171-09-11-00.wav.
Processed S0171-10-14-03.wav.
Processed S0171-10-14-06.wav.
Processed S0171-10-14-00.wav.
Processed S0171-10-14-02.wav.
Processed S0171-10-14-08.wav.
Processed S0171-10-14-07.wav.
Processed S0171-10-14-01.wav.
Processed S0171-10-14-04.wav.
Processed S0171-10-14-09.wav.
Processed S0171-10-14-05.wav.
Processed S0171-10-06-01.wav.
Processed S0171-10-06-03.wav.
Processed S0171-10-06-00.wav.
Processed S0171-10-06-05.wav.
Processed S0171-10-06-02.wav.
Processed S0171-10-06-04.wav.
Processed S0171-10-06-06.wav.
Processed S0171-10-06-08.wav.
No valid pitch values found in /content/drive/MyDrive/AIFFELthon/Data/Sample/literature_100/sample/S0171-10-06-09.wav
No pitch dat